In [34]:
import numpy as np
import os

In [35]:
COREF_LOGS = '/mnt/data0/dtadimeti/manosphere/logs/'
COREF_REDDIT = COREF_LOGS + 'coref_reddit/'

In [36]:
fem = set(['she', 'her', 'hers', 'herself'])
masc = set(['he', 'him', 'his', 'himself'])
ratios = {} # {year-month : fem frac}
all_fem_count = 0 # total
all_masc_count = 0 # total
boyfriend_lines = [] # clusters that contain 'boyfriend' substring
for year_month in os.listdir(COREF_REDDIT): 
    with open(COREF_REDDIT + year_month, 'r') as infile: 
        for line in infile: 
            # very crude check
            if 'boyfriend' in line: 
                contents = line.split('\t')
                for clust in contents: 
                    if 'boyfriend' in clust: 
                        boyfriend_lines.append(clust)
    fem_count = 0
    masc_count = 0
    for clust in boyfriend_lines: 
        items = set(clust.split('$'))
        # is a cluster that contains 'boyfriend' as a span or proceeded by one other token
        is_boyfriend_cluster = False
        for item in items: 
            if item.split(' ')[-1] == 'boyfriend' and len(item.split(' ')) <= 2: 
                is_boyfriend_cluster = True
        if is_boyfriend_cluster: 
            # only count it once per cluster
            if items & fem: 
                fem_count += 1
            # only count it once per cluster
            if items & masc: 
                masc_count += 1
    if (fem_count + masc_count) != 0: 
        ratios[year_month] = fem_count / (fem_count + masc_count)
    all_fem_count += fem_count
    all_masc_count += masc_count
print(ratios)

{'2011-07': 0.45, '2008-02': 0.45, '2011-02': 0.3333333333333333, '2016-05': 0.24175824175824176, '2019-10': 0.2037037037037037, '2006-08': 0.2037037037037037, '2012-11': 0.20829315332690454, '2015-08': 0.210989010989011, '2018-04': 0.21529745042492918, '2019-03': 0.21047996477322764, '2014-01': 0.2106490478041197, '2014-03': 0.20722300140252453, '2018-11': 0.20685434516523868, '2018-10': 0.20566603105420866, '2018-09': 0.20555692156380626, '2008-01': 0.20555692156380626, '2012-07': 0.20770321361058602, '2017-07': 0.20793374019180472, '2013-07': 0.20762100926879506, '2015-05': 0.20875290472501937, '2018-01': 0.20515097690941386, '2006-02': 0.20515097690941386, '2017-12': 0.20457180718900117, '2013-08': 0.20395156150414276, '2011-04': 0.20655427761147, '2006-06': 0.20655427761147, '2006-05': 0.20655427761147, '2013-12': 0.2046853881964259, '2010-05': 0.2049745280191789, '2005-12': 0.2049745280191789, '2015-07': 0.20427178899082568, '2007-11': 0.20427178899082568, '2016-09': 0.2045828759

In [41]:
sorted(list(ratios.values()))

[0.20349907918968693,
 0.20364820846905538,
 0.20364820846905538,
 0.2037037037037037,
 0.2037037037037037,
 0.2037805944055944,
 0.20395156150414276,
 0.20396564789088154,
 0.20396564789088154,
 0.20409452799191205,
 0.20409933846654377,
 0.2041281499303533,
 0.20427178899082568,
 0.20427178899082568,
 0.2044433939966911,
 0.20457180718900117,
 0.204582875960483,
 0.20462543991955756,
 0.2046853881964259,
 0.2049745280191789,
 0.2049745280191789,
 0.20504048352149618,
 0.20504849539915443,
 0.20509633312616532,
 0.20509633312616532,
 0.20510708401976935,
 0.20510708401976935,
 0.20515097690941386,
 0.20515097690941386,
 0.20515374081912113,
 0.2053070885760558,
 0.20549252465274095,
 0.20555692156380626,
 0.20555692156380626,
 0.20559889857732905,
 0.20563035495716034,
 0.205649894357931,
 0.20566603105420866,
 0.20652341938475807,
 0.20652341938475807,
 0.20653536257833482,
 0.20655427761147,
 0.20655427761147,
 0.20655427761147,
 0.20657026212369797,
 0.20657234957020057,
 0.2065894

In [42]:
all_fem_count / (all_fem_count + all_masc_count)

0.2077390136779212